In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchvision
import copy 
import time
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.optim import lr_scheduler
from torchvision import transforms, datasets, models
from torch.utils.tensorboard import SummaryWriter
%matplotlib

plt.ion()

data_transforms = {
    'train' : transforms.Compose([transforms.RandomResizedCrop(244), 
                                 transforms.RandomHorizontalFlip(),
                                 transforms.ToTensor(),
                                 transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])]),
    'val' : transforms.Compose([transforms.RandomResizedCrop(244), 
                               transforms.RandomHorizontalFlip(),
                               transforms.ToTensor(),
                               transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
}

data_dir = 'hymenoptera_data'
#会自动给不同的种类分别命名label为0， 1 ......
image_datasets = {x : datasets.ImageFolder(os.path.join(data_dir, x), transform = data_transforms[x]) for x in ['train', 'val']}
dataloader = {x : DataLoader(image_datasets[x], batch_size = 4, shuffle = True) for x in ['train', 'val']}
data_sizes = {x : len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

def imshow(inp, title = None) :
    #tenso->numpy需要轴转换
    inp = inp.numpy().transpose((1, 2, 0))
    
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    
    #还原原来的数据分布，因为上面transforms时有归一化
    inp = std * inp + mean
    #np.clip是一个截取函数，用于截取数组中小于或者大于某值的部分，并使得被截取部分等于固定值
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001) # pause a bit so that plots are updated

# print(class_names)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 获取一批训练数据
input, classes = next(iter(dataloader['train']))
# 批量制作网格
out = torchvision.utils.make_grid(input, padding = 0)
#imshow(out, title = [class_names[x] for x in classes])

def visualize_model(model, num_images = 6) :
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()
    
    with torch.no_grad() :
        for i, (inputs, labels) in enumerate(dataloader['val']) :
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            _, pred = torch.max(outputs, 1) 
            
            for j in range(inputs.size()[0]) :
                images_so_far += 1
                ax = plt.subplot(num_images // 2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predict : {}'.format(class_names[pred[j]]))
                a = inputs.cpu().data[j]
                imshow(inputs.cpu().data[j])
                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)
                
def train_model(model, criterion, optimizer, scheduler, num_epoch = 5) :
    since = time.time()
    best_model = copy.deepcopy(model.state_dict())
    best_acc =0.0
    
    for epoch in range(num_epoch) :
        print(f'epoch : {epoch} / {num_epoch - 1}')
        print('-' * 10)
        
        for phase in ['train', 'val'] :
            if phase == 'train' :
                scheduler.step()
                model.train()
            else :
                model.eval()
                
            running_loss = 0.0
            running_correct = 0.0
            
            for inputs, label in dataloader[phase] :
                inputs, label = inputs.to(device), label.to(device)
                #参数梯度零化
                optimizer.zero_grad()
                #torch.set_grad_enabled接受一个布尔值，若接受为True则跟踪求导，否则不追踪
                with torch.set_grad_enabled(phase == 'train') :
                    outputs = model(inputs)
                    _, pred = torch.max(outputs, 1)
                    loss = criterion(outputs, label)
                    if phase == 'train' :
                        loss.backward()
                        optimizer.step()
                #统计
                running_loss += loss.item()
                running_correct += torch.sum(pred == label.data)
            epoch_loss = running_loss / data_sizes[phase]
            epoch_acc = running_correct.double() / data_sizes[phase]
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
             # 深度复制mo
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
        print()
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    # 加载最佳模型权重
    model.load_state_dict(best_model_wts)
    return model
#场景1：微调ConvNet
#加载预训练模型并重置最终完全连接的图层。
# model_ft = models.resnet18(pretrained = True)
# num_ftrs = model_ft.fc.in_features#获取全连接层输入特征
# model_ft.fc = nn.Linear(num_ftrs, 2)#重置全连接层

# model_ft = model_ft.to(device)
# criterion = nn.CrossEntropyLoss()
# optimizer_ft = optim.SGD(model_ft.parameters(), lr = 0.001, momentum = 0.9)

# # 每7个epochs衰减LR通过设置gamma=0.1
# exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size = 7, gamma = 0.1)
# model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epoch = 5)

# visualize_model(model_ft)

#.场景2：ConvNet作为固定特征提取器
#在这里需要冻结除最后一层之外的所有网络。通过设置 requires_grad == Falsebackward() 来冻结参数，这样在反向传播backward()的时候他们的梯度就不会被计算
model_conv = models.resnet18(pretrained = True)

for param in model_conv.parameters() :
    param.requires_gradv = False

num_ftrs = model_conv.fc.in_features
model_conv.fc = nn.Linear(num_ftrs, 2)
model_conv = model_conv.to(device)

criterion = nn.CrossEntropyLoss()
optimizer_conv = optim.SGD(model_conv.fc.parameters(), lr = 0.001, momentum = 0.9)

exp_lr_scheduler =lr_scheduler.StepLR(optimizer_conv, step_size=7, gamma=0.1)

model_conv = train_model(model_conv, criterion, optimizer_conv, exp_lr_scheduler, num_epoch = 5)

visualize_model(model_conv)
plt.ioff()
plt.show()

ModuleNotFoundError: No module named 'tensorboard'